## Main

In [ ]:
import gc
import copy
import math
import random
import numpy as np
from datetime import datetime
import time
from tqdm import tqdm
import pandas as pd
from typing import Dict, Any, Tuple
import optuna
from optuna.pruners import HyperbandPruner#, SuccessiveHalvingPruner
from optuna.integration import CatBoostPruningCallback
from optuna.integration import XGBoostPruningCallback
from optuna.integration import LightGBMPruningCallback
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn import set_config
from modules.labeling_lib import get_prices
from modules.labeling_lib import get_features
from modules.labeling_lib import get_labels_one_direction
from modules.labeling_lib import sliding_window_clustering
from modules.tester_lib import robust_oos_score_one_direction
from modules.tester_lib import walk_forward_score_one_direction
from modules.export_lib import export_model_to_ONNX
from modules.export_lib import XGBWithEval
from modules.export_lib import LGBMWithEval
from modules.export_lib import CatWithEval
import warnings
warnings.filterwarnings("ignore")
set_config(enable_metadata_routing=True, skip_parameter_validation=True)
optuna.logging.set_verbosity(optuna.logging.WARNING)

def fit_final_models(main_data: pd.DataFrame,
                    meta_data: pd.DataFrame,
                    ds_test: pd.DataFrame,
                    ds_train: pd.DataFrame,
                    hp: Dict[str, Any],
                    trial: optuna.trial.Trial) -> Tuple[float, float, Any, Any]:
    def check_constant_features(X):
        return np.any(np.var(X, axis=0) < 1e-10)
    
    gc.collect()
    # ---------- 1) main model_main ----------
    X_main = main_data.loc[:, main_data.columns.str.contains('_feature') & ~main_data.columns.str.contains('_meta_feature')]
    if X_main.shape[1] == 1:
        if check_constant_features(X_main.to_numpy()):
            return None, None, None, None
    y_main = main_data['labels'].astype('int16')
    # Check for inf values in main features
    inf_cols_main = X_main.columns[X_main.isin([np.inf, -np.inf]).any()].tolist()
    if inf_cols_main:
        print("Main features with inf values:", inf_cols_main)
    # Check for NaN values in main features
    nan_cols_main = X_main.columns[X_main.isna().any()].tolist()
    if nan_cols_main:
        print("Main features with NaN values:", nan_cols_main)
    # División de datos para el modelo principal según fechas
    X_train_main, X_val_main, y_train_main, y_val_main = train_test_split(
        X_main, y_main, 
        test_size=0.2,
        shuffle=True
    )
    # ── descartar clusters problemáticos ────────────────────────────
    if len(y_train_main.value_counts()) < 2 or len(y_val_main.value_counts()) < 2:
        return None, None, None, None
    
    # ---------- 2) meta‑modelo ----------
    X_meta = meta_data.loc[:, meta_data.columns.str.contains('_meta_feature')]
    if X_meta.shape[1] == 1:
        if check_constant_features(X_meta.to_numpy()):
            return None, None, None, None
    y_meta = meta_data['clusters'].astype('int16')
    # Check for inf values in meta features
    inf_cols_meta = X_meta.columns[X_meta.isin([np.inf, -np.inf]).any()].tolist()
    if inf_cols_meta:
        print("Meta features with inf values:", inf_cols_meta)
    # Check for NaN values in meta features
    nan_cols_meta = X_meta.columns[X_meta.isna().any()].tolist()
    if nan_cols_meta:
        print("Meta features with NaN values:", nan_cols_meta)
    # División de datos para el modelo principal según fechas
    X_train_meta, X_val_meta, y_train_meta, y_val_meta = train_test_split(
        X_meta, y_meta, 
        test_size=0.2,
        shuffle=True
    )
    # ── descartar clusters problemáticos ────────────────────────────
    if len(y_train_meta.value_counts()) < 2 or len(y_val_meta.value_counts()) < 2:
        return None, None, None, None

    # print(f"Main columns: {X_main.columns.tolist()}")
    # print(f"Meta columns: {X_meta.columns.tolist()}")
    
    # Main model
    cat_main_params = dict(
        iterations=hp['cat_main_iterations'],
        depth=hp['cat_main_depth'],
        learning_rate=hp['cat_main_learning_rate'],
        l2_leaf_reg=hp['cat_main_l2_leaf_reg'],
        early_stopping_rounds=hp['cat_main_early_stopping'],
        eval_metric='Accuracy',
        verbose=False,
        thread_count=-1,
        task_type='CPU',
        used_ram_limit="16gb"
    )
    xgb_main_params = dict(
        n_estimators=hp['xgb_main_estimators'],
        max_depth=hp['xgb_main_max_depth'],
        learning_rate=hp['xgb_main_learning_rate'],
        reg_lambda=hp['xgb_main_reg_lambda'],
        early_stopping_rounds=hp['xgb_main_early_stopping'],
        eval_metric='logloss',
        verbosity=0,
        n_jobs=-1,
        tree_method= "gpu_hist",
        device_type="cuda"
    )
    lgbm_main_params = dict(
        n_estimators=hp['lgbm_main_estimators'],
        max_depth=hp['lgbm_main_max_depth'],
        learning_rate=hp['lgbm_main_learning_rate'],
        reg_lambda=hp['lgbm_main_reg_lambda'],
        early_stopping_round=hp['lgbm_main_early_stopping'],
        metric='auc',
        tree_learner="serial",
        device="cpu",
        verbosity=-1,
        n_jobs=-1
    )
    base_main_models = [
        ('catboost', CatWithEval(
            **cat_main_params,
            eval_set=[(X_val_main, y_val_main)],
            callbacks=[CatBoostPruningCallback(trial, "Accuracy")])),
        ('xgboost', XGBWithEval(
            **xgb_main_params, 
            eval_set=[(X_val_main, y_val_main)],
            callbacks=[XGBoostPruningCallback(trial, "validation_0-logloss")]
        )),
        ('lightgbm', LGBMWithEval(
            **lgbm_main_params, 
            eval_set=[(X_val_main, y_val_main)],
            callbacks=[LightGBMPruningCallback(trial, "auc")])),
    ]
    model_main = VotingClassifier(
            estimators=base_main_models,
            voting='soft',
            flatten_transform=False,
            n_jobs=1
        )
    # print("training main model...")
    # start_time = time.time()
    model_main.fit(X_train_main, y_train_main)
    #print(f"main model trained in {time.time() - start_time:.2f} seconds")

    gc.collect()

    # Meta-modelo
    cat_meta_params = dict(
        iterations=hp['cat_meta_iterations'],
        depth=hp['cat_meta_depth'],
        learning_rate=hp['cat_meta_learning_rate'],
        l2_leaf_reg=hp['cat_meta_l2_leaf_reg'],
        early_stopping_rounds=hp['cat_meta_early_stopping'],
        eval_metric='F1',
        verbose=False,
        thread_count=-1,
        task_type='CPU',
        used_ram_limit="16gb"
    )
    xgb_meta_params = dict(
        n_estimators=hp['xgb_meta_estimators'],
        max_depth=hp['xgb_meta_max_depth'],
        learning_rate=hp['xgb_meta_learning_rate'],
        reg_lambda=hp['xgb_meta_reg_lambda'],
        early_stopping_rounds=hp['xgb_meta_early_stopping'],
        eval_metric='auc',
        verbosity=0,
        verbose_eval=False,
        n_jobs=-1,
        tree_method= "gpu_hist",
        device_type="cuda"
    )
    lgbm_meta_params = dict(
        n_estimators=hp['lgbm_meta_estimators'],
        max_depth=hp['lgbm_meta_max_depth'],
        learning_rate=hp['lgbm_meta_learning_rate'],
        reg_lambda=hp['lgbm_meta_reg_lambda'],
        early_stopping_round=hp['lgbm_meta_early_stopping'],
        metric='binary_logloss',
        tree_learner="serial",
        device="cpu",
        verbosity=-1,
        n_jobs=-1
    )
    base_meta_models = [
        ('catboost', CatWithEval(
            **cat_meta_params,
            eval_set=[(X_val_meta, y_val_meta)],
            callbacks=[CatBoostPruningCallback(trial, "F1")])),
        ('xgboost', XGBWithEval(
            **xgb_meta_params, 
            eval_set=[(X_val_meta, y_val_meta)],
            callbacks=[XGBoostPruningCallback(trial, "validation_0-auc")])),
        ('lightgbm', LGBMWithEval(
            **lgbm_meta_params, 
            eval_set=[(X_val_meta, y_val_meta)],
            callbacks=[LightGBMPruningCallback(trial, "binary_logloss")])),
    ]

    model_meta = VotingClassifier(
            estimators=base_meta_models,
            voting='soft',
            flatten_transform=False,
            n_jobs=1
        )
    # print("training meta model...")
    # start_time = time.time()
    model_meta.fit(X_train_meta, y_train_meta)
    # print(f"meta model trained in {time.time() - start_time:.2f} seconds")

    # ── evaluación ───────────────────────────────────────────────
    # print("evaluating in-sample...")
    # start_time = time.time()
    r2_ins = robust_oos_score_one_direction(
        ds_train,
        [model_main, model_meta],
        direction = hp['direction'],
        n_sim = 100, mc_mode="both", agg="q05"
    )
    # print(f"in-sample score calculated in {time.time() - start_time:.2f} seconds")
    # print("evaluating out-of-sample...")
    # start_time = time.time()
    r2_oos = walk_forward_score_one_direction(
        ds_test,
        [model_main, model_meta],
        direction     = hp['direction'],
        n_sim = 100, mc_mode="both",
        agg       = "q05",
        final_agg = "median"
    )
    # print(f"out-of-sample score calculated in {time.time() - start_time:.2f} seconds /n")
    return r2_ins, r2_oos, model_main, model_meta

def objective(trial: optuna.trial.Trial, base_hp: Dict[str, Any], study=None) -> float:
    def calc_score(fwd: float, bwd: float, eps: float = 1e-9) -> float:
        if (fwd is None or bwd is None or
            not np.isfinite(fwd) or not np.isfinite(bwd) or
            fwd <= 0 or bwd <= 0):
            return -1.0
        mean = 0.4 * fwd + 0.6 * bwd
        if fwd < bwd * 0.8:
            mean *= 0.8
        delta  = abs(fwd - bwd) / max(abs(fwd), abs(bwd), eps)
        score  = mean * (1.0 - delta)
        return score
    
    def sample_cat_params(trial, prefix: str):
        iterations = trial.suggest_int(f"{prefix}_iterations", 100, 300, step=50)
        depth      = trial.suggest_int(f"{prefix}_depth", 3, 6)
        lr         = trial.suggest_float(f"{prefix}_learning_rate", 0.15, 0.3, log=True)
        l2         = trial.suggest_float(f"{prefix}_l2_leaf_reg", 1.0, 5.0, log=True)
        es_rounds  = trial.suggest_int(f"{prefix}_early_stopping", 30, 60, step=10)
        return {
            f"{prefix}_iterations": iterations,
            f"{prefix}_depth": depth,
            f"{prefix}_learning_rate": lr,
            f"{prefix}_l2_leaf_reg": l2,
            f"{prefix}_early_stopping": es_rounds,
        }
    
    def sample_xgb_params(trial, prefix: str):
        n_estimators = trial.suggest_int(f"{prefix}_estimators", 100, 300, step=50)
        max_depth = trial.suggest_int(f"{prefix}_max_depth", 3, 6)
        lr           = trial.suggest_float(f"{prefix}_learning_rate", 0.15, 0.3, log=True)
        reg_lambda   = trial.suggest_float(f"{prefix}_reg_lambda", 1.0, 5.0, log=True)
        es_rounds    = trial.suggest_int(f"{prefix}_early_stopping", 30, 60, step=10)
        return {
            f"{prefix}_estimators": n_estimators,
            f"{prefix}_max_depth": max_depth,
            f"{prefix}_learning_rate": lr,
            f"{prefix}_reg_lambda": reg_lambda,
            f"{prefix}_early_stopping": es_rounds,
        }

    def sample_lgbm_params(trial, prefix: str):
        n_estimators = trial.suggest_int(f"{prefix}_estimators", 100, 300, step=50)
        max_depth    = trial.suggest_int(f"{prefix}_max_depth", 3, 6)
        lr           = trial.suggest_float(f"{prefix}_learning_rate", 0.15, 0.3, log=True)
        reg_lambda   = trial.suggest_float(f"{prefix}_reg_lambda", 1.0, 5.0, log=True)
        es_rounds    = trial.suggest_int(f"{prefix}_early_stopping", 30, 60, step=10)
        return {
            f"{prefix}_estimators": n_estimators,
            f"{prefix}_max_depth": max_depth,
            f"{prefix}_learning_rate": lr,
            f"{prefix}_reg_lambda": reg_lambda,
            f"{prefix}_early_stopping": es_rounds,
        }

    gc.collect()
    best_combined_score = -math.inf
    hp = {k: copy.deepcopy(v) for k, v in base_hp.items() if k != 'base_df'}

    # µ··· Espacio de búsqueda optimizado ···µ
    hp['n_clusters'] = trial.suggest_int('n_clusters', 5, 50, step=5)
    hp['k'] = trial.suggest_int('k', 2, 10, step=1)
    hp['atr_period'] = trial.suggest_int('atr_period', 5, 50, step=5)
    hp['markup'] = trial.suggest_float("markup", 0.1, 1.0, step=0.1)
    hp['label_max'] = trial.suggest_int('label_max', 2, 6, step=1, log=True)
    #hp['step'] = trial.suggest_int('step', 1, hp['label_max'])

    # Optimización de hiperparámetros
    hp.update(sample_cat_params(trial, "cat_main"))
    hp.update(sample_cat_params(trial, "cat_meta"))
    hp.update(sample_xgb_params(trial, "xgb_main"))
    hp.update(sample_xgb_params(trial, "xgb_meta"))
    hp.update(sample_lgbm_params(trial, "lgbm_main"))
    hp.update(sample_lgbm_params(trial, "lgbm_meta"))

    # Optimización de períodos para el modelo principal
    n_periods_main = trial.suggest_int('n_periods_main', 5, 15, log=True)
    main_periods = []
    for i in range(n_periods_main):
        period_main = trial.suggest_int(f'period_main_{i}', 2, 200, log=True)
        main_periods.append(period_main)
    main_periods = sorted(list(set(main_periods)))
    hp['periods_main'] = main_periods
    # Selección de estadísticas para el modelo principal
    main_stat_choices = [
        "std", "skew", "kurt", "zscore", "range", "mad", "entropy", 
        "slope", "momentum", "fractal", "hurst", "autocorr", "max_dd", 
        "sharpe", "fisher", "chande", "var", "approx_entropy", 
        "eff_ratio", "corr_skew", "jump_vol", "vol_skew", "hurst"
    ]
    n_main_stats = trial.suggest_int('n_main_stats', 1, 5, log=True)
    selected_main_stats = []
    for i in range(n_main_stats):
        stat = trial.suggest_categorical(f'main_stat_{i}', main_stat_choices)
        selected_main_stats.append(stat)
    selected_main_stats = list(set(selected_main_stats))
    hp["stats_main"] = selected_main_stats
    #print(f"Main features seleccionadas: {hp['stats_main']}")

    # Optimización de períodos para el meta-modelo
    n_periods_meta = 1 # trial.suggest_int('n_periods_meta', 1, 3, log=True)
    meta_periods = []
    for i in range(n_periods_meta):
        period_meta = trial.suggest_int(f'period_meta_{i}', 2, 6, log=True)
        meta_periods.append(period_meta)
    meta_periods = sorted(list(set(meta_periods)))
    hp['periods_meta'] = meta_periods
    # Selección de estadísticas para el meta-modelo
    meta_stat_choices = [
        "std", "skew", "kurt", "zscore", "range", "mad", "entropy", 
        "slope", "momentum", "fractal", "hurst", "autocorr", "max_dd", 
        "sharpe", "fisher", "chande", "var", "approx_entropy", 
        "eff_ratio", "corr_skew", "jump_vol", "vol_skew", "hurst"
    ]
    n_meta_stats = trial.suggest_int('n_meta_stats', 1, 3, log=True)
    selected_meta_stats = []
    for i in range(n_meta_stats):
        stat = trial.suggest_categorical(f'meta_stat_{i}', meta_stat_choices)
        selected_meta_stats.append(stat)
    selected_meta_stats = list(set(selected_meta_stats))
    hp["stats_meta"] = selected_meta_stats
    #print(f"Meta features seleccionadas: {hp['stats_meta']} | Periodo: {hp['periods_meta']}")

    # Dataset completo obtener caracteristicas
    full_ds = get_features(base_hp['base_df'], hp)
    # Seccionar dataset de entrenamiento
    test_mask  = (full_ds.index >= hp["test_start"]) & (full_ds.index <= hp["test_end"])
    train_mask = (full_ds.index >= hp["train_start"]) & (full_ds.index <= hp["train_end"]) & ~test_mask
    ds_train = full_ds[train_mask]
    ds_test  = full_ds[test_mask]

    # Clustering
    ds_train = sliding_window_clustering(
        ds_train,
        n_clusters=hp['n_clusters'],
        step=hp.get('step', None),
        atr_period=hp['atr_period'],
        k=hp['k']
    )
    # Evaluar clusters ordenados por tamaño
    cluster_sizes = ds_train['clusters'].value_counts()
    # Filtrar el cluster 0 (inválido) si existe
    if 0 in cluster_sizes.index:
        cluster_sizes = cluster_sizes.drop(0)
    for clust in cluster_sizes.index:
        # Main data
        main_data = ds_train.loc[ds_train['clusters'] == clust]
        if len(main_data) <= hp['label_max']:
            continue
        main_data = get_labels_one_direction(
            main_data,
            markup=hp['markup'],
            max_val=hp['label_max'],
            direction=hp['direction'],
            atr_period=hp['atr_period'],
            deterministic=True
        )
        if (main_data['labels'].value_counts() < 2).any():
            continue

        # Meta data
        meta_data = ds_train.copy()
        meta_data['clusters'] = (meta_data['clusters'] == clust).astype(int)
        if (meta_data['clusters'].value_counts() < 2).any():
            continue

        # Evaluación en ambos períodos
        r2_ins, r2_oos, model_main, meta_model = fit_final_models(
            main_data,
            meta_data,
            ds_train,
            ds_test,
            hp,
            trial
        )
        if r2_ins == None or r2_oos == None or model_main == None or meta_model == None:
            continue
        # Calcular puntuación combinada (puedes ajustar los pesos según necesites)
        score = calc_score(r2_ins, r2_oos)
        if score <= -1.0:
             continue

        if score > best_combined_score:
            best_combined_score = score
            # Guardar información del trial actual
            trial.set_user_attr("r2_ins", r2_ins)
            trial.set_user_attr("r2_oos", r2_oos)
            trial.set_user_attr("combined_score", score)
            trial.set_user_attr("cluster_id", clust)
            trial.set_user_attr("stats_main", hp["stats_main"])
            trial.set_user_attr("stats_meta", hp["stats_meta"])
            trial.set_user_attr("periods_main", hp["periods_main"])
            trial.set_user_attr("periods_meta", hp["periods_meta"])
            # Guardar parámetros del trial actual (sin fechas)
            params_to_save = hp.copy()
            params_to_save.pop('backward', None)
            params_to_save.pop('forward', None)
            params_to_save.pop('full_forward', None)
            trial.set_user_attr("params", params_to_save)
            # Si existe el estudio, actualizar sus atributos
            if study is not None:
                current_best = study.user_attrs.get("best_combined_score", -math.inf)
                if score > current_best:
                    study.set_user_attr("best_params", params_to_save)
                    study.set_user_attr("best_metrics", {
                        "r2_ins": r2_ins,
                        "r2_oos": r2_oos,
                        "combined_score": score,
                        "cluster_id": clust,
                    })
                    study.set_user_attr("best_combined_score", score)
                    study.set_user_attr("best_models", [model_main, meta_model])
                    study.set_user_attr("best_stats_main", hp["stats_main"])
                    study.set_user_attr("best_stats_meta", hp["stats_meta"])
                    study.set_user_attr("best_periods_main", hp["periods_main"])
                    study.set_user_attr("best_periods_meta", hp["periods_meta"])
                    study.set_user_attr("best_trial_number", trial.number)
                    study.set_user_attr("best_df_sample", full_ds.sample(1))
    # Si no hay ningún cluster válido, devolver un valor negativo pero no infinito
    if best_combined_score == -math.inf:
        return -1.0

    # No aplicar penalización adicional por pocos clusters para mantener coherencia
    return best_combined_score

def optimize_and_export(base_hp: Dict[str, Any]):
    """Lanza Optuna, guarda el mejor modelo y lo exporta a ONNX."""
    def show_best_summary(study: optuna.study.Study, model_seed) -> None:
        """Muestra en un único cuadro el resultado óptimo del estudio."""
        # ── extraer métricas y modelos ───────────────────────────────────────
        m  = study.user_attrs.get("best_metrics", {})
        f2 = m.get("r2_ins",   float("nan"))
        b2 = m.get("r2_oos",  float("nan"))
        c2 = m.get("combined_score", float("nan"))
        best_trial = study.user_attrs.get("best_trial_number", "-")
        best_df_sample = study.user_attrs.get("best_df_sample", None)

        main_cls = meta_cls = "None"
        best_models = study.user_attrs.get("best_models")
        if best_models and len(best_models) == 2:
            main_cls = type(best_models[0]).__name__
            meta_cls = type(best_models[1]).__name__

        # ── cuadro de resumen ────────────────────────────────────────────────
        lines = [
            "┌" + "─" * 55 + "┐",
            f"│  MODELO {model_seed} TRIAL ÓPTIMO #{best_trial}",
            "├" + "─" * 55 + "┤",
            f"│  R² INS : {f2:10.4f}  │  R² OOS : {b2:10.4f} │",
            f"│  Combined     : {c2:10.4f}                            │",
            "├" + "─" * 55 + "┤"
        ]
        print("\n".join(lines))

    base_hp.update({
        'model_seed': random.randint(0, 10000000),
    })
    # Configurar el pruner inteligente
    # pruner = SuccessiveHalvingPruner(
    #     min_resource=1,
    #     reduction_factor=3,
    #     min_early_stopping_rate=0
    # )

    # Crear el estudio sin persistencia
    study = optuna.create_study(
        direction='maximize',
        pruner=HyperbandPruner(),
        sampler=optuna.samplers.TPESampler(
            n_startup_trials=int(np.sqrt(base_hp['n_trials'])),
        )
    )
    study.optimize(lambda t: objective(t, base_hp, study),
                n_trials=base_hp['n_trials'],
                show_progress_bar=True)

    # ── verificación y cuadro ───────────────────────────────
    best_trial = study.best_trial
    assert study.user_attrs.get("best_trial_number") == best_trial.number, \
        "best_trial_number no coincide con study.best_trial.number"

    best_metric_saved = study.user_attrs.get("best_metrics", {}).get("combined_score")
    if best_metric_saved is not None:
        assert math.isclose(best_metric_saved, study.best_value, rel_tol=1e-9), \
            "combined_score guardado ≠ study.best_value"
    else:
        print("⚠️  Ningún trial produjo un score válido; modelos no guardados.")

    best_models = study.user_attrs.get("best_models")
    assert best_models and len(best_models) == 2 and all(best_models), \
        "best_models incorrecto"

    show_best_summary(study, base_hp['model_seed'])

    # ── exportación ─────────────────────────────────────────
    export_params = base_hp.copy()
    export_params.update({
        "best_trial": study.user_attrs["best_trial_number"],
        "best_score": study.user_attrs["best_combined_score"],
        "best_periods_main": study.user_attrs["best_periods_main"],
        "best_periods_meta": study.user_attrs["best_periods_meta"],
        "best_stats_main"  : study.user_attrs["best_stats_main"],
        "best_stats_meta"  : study.user_attrs["best_stats_meta"],
        "best_models"      : study.user_attrs["best_models"],
    })
    export_model_to_ONNX(**export_params)
    # ── devolver métricas ─────────────────────────────────────────
    return {
        "r2_ins"   : study.user_attrs.get("best_metrics", {}).get("r2_ins",   float("nan")),
        "r2_oos"  : study.user_attrs.get("best_metrics", {}).get("r2_oos",  float("nan")),
        "combined_score": study.user_attrs.get("best_metrics", {}).get("combined_score", float("nan")),
    }

if __name__ == "__main__":
    base_hp: Dict[str, Any] = {
        'symbol': r'XAUUSD',
        'timeframe': 'H1',
        'direction': 'buy',
        'train_start': datetime(2019, 1, 1),
        'train_end': datetime(2025, 1, 1),
        'test_start': datetime(2022, 1, 1),
        'test_end': datetime(2023, 1, 1),
        'model_seed': 0,
        'n_trials': 1000,
        'models_export_path': r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files/',
        'include_export_path': r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Include/ajmtrz/include/Dmitrievsky',
        'history_path': r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/",
        'stats_main': [],
        'stats_meta': [],
        'best_models': [None, None],
        'markup': 0.20,
        'label_min'  : 1,
        'label_max'  : 15,
        'n_clusters': 30,
        'window_size': 350,
        'periods_main': [i for i in range(5, 300, 30)],
        'periods_meta': [5],
    }
    base_df = get_prices(base_hp)
    base_hp.update({
        'base_df': base_df,
    })
    # Para recopilar resultados globales de todos los modelos
    all_results = {}
    best_models = []
    model_range = range(0, 5)
    base_hp.update({'n_trials': 500})
    for i in tqdm(model_range, desc=f"Optimizando {base_hp['symbol']}/{base_hp['timeframe']}", unit="modelo"):
        try:
            model_results = optimize_and_export(base_hp)
            best_models.append((i, model_results))
            
            # Añadir a resultados globales
            all_results[f"model_{i}"] = {
                "success": True,
                "r2_ins": model_results["r2_ins"],
                "r2_oos": model_results["r2_oos"],
                "combined_score": model_results["combined_score"]
            }
            
        except Exception as e:
            import traceback
            tqdm.write(f"\nError procesando modelo {i}: {str(e)}")
            tqdm.write(traceback.format_exc())
            
            all_results[f"model_{i}"] = {
                "success": False,
                "error": str(e)
            }
            continue
    
    # Resumen final
    print("\n" + "="*50)
    print(f"RESUMEN DE OPTIMIZACIÓN {base_hp['symbol']}/{base_hp['timeframe']}")
    print("="*50)
    
    successful_models = [info for model_key, info  in all_results.items() if info.get("success", False)]
    print(f"Modelos completados exitosamente: {len(successful_models)}/{len(model_range)}")
    
    if successful_models:
        # Calcular estadísticas globales
        r2_ins_scores = [info["r2_ins"] for info in successful_models]
        r2_oos_scores = [info["r2_oos"] for info in successful_models]
        combined_scores = [info["combined_score"] for info in successful_models]
        
        print(f"\nEstadísticas de rendimiento:")
        print(f"  R2 INS promedio: {np.mean(r2_ins_scores):.4f} ± {np.std(r2_ins_scores):.4f}")
        print(f"  R2 OOS promedio: {np.mean(r2_oos_scores):.4f} ± {np.std(r2_oos_scores):.4f}")
        print(f"  Puntuación combinada promedio: {np.mean(combined_scores):.4f} ± {np.std(combined_scores):.4f}")

        # Identificar el mejor modelo global basado en la puntuación combinada
        successful = [(k, v) for k, v in all_results.items() if v.get("success", False)]
        combined_scores = [v["combined_score"] for _, v in successful]
        best_model_key, best_info = successful[int(np.argmax(combined_scores))]
        
        print(f"\nMejor modelo global: {best_model_key}")
        print(f"  R2 INS: {best_info['r2_ins']:.4f}")
        print(f"  R2 OOS: {best_info['r2_oos']:.4f}")
        print(f"  Puntuación combinada: {best_info['combined_score']:.4f}")
    
    print("\nProceso de optimización completado.")

Optimizando XAUUSD/H1:   0%|          | 0/5 [00:00<?, ?modelo/s]

  0%|          | 0/500 [00:00<?, ?it/s]